In [1]:
# example of a wgan for generating handwritten digits
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras import backend
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from matplotlib import pyplot

In [2]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value

	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)

	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [3]:
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

In [4]:
# define the standalone critic model
def define_critic(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# weight constraint
	const = ClipConstraint(0.01)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const, input_shape=in_shape))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# scoring, linear activation
	model.add(Flatten())
	model.add(Dense(1))
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

In [5]:
# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

In [6]:
# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic):
	# make weights in the critic not trainable
	for layer in critic.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the critic
	model.add(critic)
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

In [7]:
# load images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# select all of the examples for a given class
	selected_ix = trainy == 7
	X = trainX[selected_ix]
	# expand to 3d, e.g. add channels
	X = expand_dims(X, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

In [8]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels, -1 for 'real'
	y = -ones((n_samples, 1))
	return X, y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [10]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels with 1.0 for 'fake'
	y = ones((n_samples, 1))
	return X, y

In [11]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

In [12]:
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
	# plot history
	pyplot.plot(d1_hist, label='crit_real')
	pyplot.plot(d2_hist, label='crit_fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	pyplot.savefig('plot_line_plot_loss.png')
	pyplot.close()

In [13]:
# train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for keeping track of loss
	c1_hist, c2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update critic model weights
			c_loss1 = c_model.train_on_batch(X_real, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update critic model weights
			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(mean(c1_tmp))
		c2_hist.append(mean(c2_tmp))
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = -ones((n_batch, 1))
		# update the generator via the critic's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		g_hist.append(g_loss)
		# summarize loss on this batch
		print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
	# line plots of loss
	plot_history(c1_hist, c2_hist, g_hist)

In [14]:
# size of the latent space
latent_dim = 50
# create the critic
critic = define_critic()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, critic, gan_model, dataset, latent_dim)

Metal device set to: Apple M1 Pro


2022-07-02 14:43:46.556050: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-02 14:43:46.556156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


/Users/manuel/miniforge3/envs/tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


11501568/11490434 [==============================] - 4s 0us/step
(6265, 28, 28, 1)


2022-07-02 14:43:51.721766: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-02 14:43:51.721851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-02 14:43:54.592868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-02 14:43:55.473317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

>Saved: generated_plot_0097.png and model_0097.h5
>98, c1=-76.799, c2=-66.400 g=-76.009
>99, c1=-76.353, c2=-67.378 g=-77.460
>100, c1=-77.721, c2=-67.865 g=-77.253
>101, c1=-78.902, c2=-68.859 g=-78.663
>102, c1=-80.042, c2=-69.219 g=-79.815
>103, c1=-79.900, c2=-70.358 g=-80.644
>104, c1=-79.870, c2=-70.966 g=-81.412
>105, c1=-81.925, c2=-71.887 g=-82.843
>106, c1=-82.710, c2=-72.341 g=-82.958
>107, c1=-83.864, c2=-73.369 g=-84.257
>108, c1=-83.751, c2=-73.979 g=-85.210
>109, c1=-84.831, c2=-74.619 g=-85.679
>110, c1=-85.555, c2=-75.645 g=-86.798
>111, c1=-85.897, c2=-76.302 g=-87.836
>112, c1=-86.939, c2=-76.870 g=-88.338
>113, c1=-87.253, c2=-77.851 g=-89.435
>114, c1=-88.156, c2=-78.397 g=-88.951
>115, c1=-89.073, c2=-79.439 g=-91.146
>116, c1=-89.499, c2=-79.981 g=-91.989
>117, c1=-90.307, c2=-80.851 g=-92.770
>118, c1=-90.846, c2=-81.628 g=-92.649
>119, c1=-92.046, c2=-82.338 g=-94.306
>120, c1=-92.278, c2=-83.069 g=-95.128
>121, c1=-93.353, c2=-83.806 g=-96.317
>122, c1=-94.007

>286, c1=-234.509, c2=-222.358 g=-244.556
>287, c1=-235.780, c2=-223.033 g=-245.889
>288, c1=-236.180, c2=-224.121 g=-246.939
>289, c1=-235.930, c2=-224.832 g=-247.512
>290, c1=-238.508, c2=-226.171 g=-248.866
>291, c1=-238.375, c2=-226.415 g=-249.778
>Saved: generated_plot_0291.png and model_0291.h5
>292, c1=-239.633, c2=-227.832 g=-250.167
>293, c1=-240.930, c2=-229.021 g=-251.287
>294, c1=-242.069, c2=-229.910 g=-252.112
>295, c1=-241.547, c2=-230.852 g=-253.364
>296, c1=-243.530, c2=-231.214 g=-254.435
>297, c1=-244.628, c2=-232.825 g=-255.473
>298, c1=-244.190, c2=-232.443 g=-256.277
>299, c1=-246.341, c2=-234.352 g=-257.617
>300, c1=-248.884, c2=-235.396 g=-258.699
>301, c1=-248.021, c2=-236.275 g=-259.072
>302, c1=-248.445, c2=-237.651 g=-260.723
>303, c1=-250.317, c2=-238.178 g=-261.821
>304, c1=-251.247, c2=-239.106 g=-262.895
>305, c1=-251.987, c2=-239.756 g=-263.534
>306, c1=-253.648, c2=-241.308 g=-264.874
>307, c1=-254.455, c2=-242.056 g=-265.734
>308, c1=-254.554, c2=-243

>472, c1=-145.818, c2=-106.532 g=-247.048
>473, c1=-149.643, c2=-89.234 g=-245.122
>474, c1=-141.190, c2=-80.693 g=-240.359
>475, c1=-130.285, c2=-50.678 g=-238.293
>476, c1=-161.438, c2=-46.401 g=-239.977
>477, c1=-148.432, c2=-29.539 g=-235.266
>478, c1=-175.954, c2=-17.235 g=-232.610
>479, c1=-172.388, c2=-8.353 g=-230.851
>480, c1=-171.597, c2=-3.017 g=-226.688
>481, c1=-190.138, c2=-2.772 g=-221.818
>482, c1=-189.867, c2=-9.961 g=-221.929
>483, c1=-205.807, c2=-18.169 g=-215.964
>484, c1=-213.238, c2=-27.627 g=-211.015
>485, c1=-224.922, c2=-41.462 g=-196.459
>Saved: generated_plot_0485.png and model_0485.h5
>486, c1=-234.496, c2=-60.799 g=-179.119
>487, c1=-250.077, c2=-79.927 g=-164.916
>488, c1=-260.834, c2=-102.144 g=-146.206
>489, c1=-270.966, c2=-124.489 g=-128.367
>490, c1=-278.590, c2=-145.649 g=-106.447
>491, c1=-289.034, c2=-167.929 g=-90.623
>492, c1=-302.353, c2=-183.683 g=-72.974
>493, c1=-301.804, c2=-198.379 g=-57.013
>494, c1=-306.836, c2=-213.169 g=-31.067
>495, c

>662, c1=-638.455, c2=-580.440 g=581.040
>663, c1=-642.512, c2=-581.733 g=582.273
>664, c1=-641.264, c2=-582.898 g=583.649
>665, c1=-644.549, c2=-584.234 g=584.803
>666, c1=-644.551, c2=-585.542 g=586.060
>667, c1=-647.078, c2=-586.680 g=587.242
>668, c1=-647.577, c2=-587.871 g=588.608
>669, c1=-648.541, c2=-589.158 g=589.794
>670, c1=-651.070, c2=-590.507 g=591.072
>671, c1=-651.885, c2=-591.627 g=592.353
>672, c1=-653.839, c2=-592.881 g=593.598
>673, c1=-655.701, c2=-594.208 g=594.864
>674, c1=-656.506, c2=-595.441 g=596.120
>675, c1=-658.183, c2=-596.722 g=597.351
>676, c1=-658.319, c2=-597.938 g=598.602
>677, c1=-660.343, c2=-599.117 g=599.839
>678, c1=-659.833, c2=-600.411 g=601.127
>679, c1=-664.141, c2=-601.510 g=602.406
>Saved: generated_plot_0679.png and model_0679.h5
>680, c1=-665.050, c2=-602.867 g=603.594
>681, c1=-665.068, c2=-604.062 g=604.888
>682, c1=-668.164, c2=-605.204 g=606.047
>683, c1=-669.626, c2=-606.502 g=607.265
>684, c1=-671.871, c2=-607.564 g=608.477
>685, c

>852, c1=-803.691, c2=-623.237 g=715.546
>853, c1=-804.373, c2=-596.352 g=711.374
>854, c1=-796.610, c2=-596.189 g=712.134
>855, c1=-798.841, c2=-614.183 g=711.085
>856, c1=-796.455, c2=-621.378 g=704.485
>857, c1=-805.243, c2=-633.766 g=717.746
>858, c1=-812.558, c2=-638.248 g=718.208
>859, c1=-814.821, c2=-659.634 g=717.603
>860, c1=-814.263, c2=-655.687 g=725.210
>861, c1=-823.171, c2=-670.039 g=721.903
>862, c1=-824.411, c2=-676.414 g=720.409
>863, c1=-827.771, c2=-680.457 g=726.240
>864, c1=-828.871, c2=-679.207 g=722.905
>865, c1=-832.367, c2=-689.327 g=726.625
>866, c1=-837.384, c2=-690.169 g=723.736
>867, c1=-836.532, c2=-684.189 g=722.266
>868, c1=-837.261, c2=-689.064 g=715.557
>869, c1=-841.679, c2=-681.140 g=712.490
>870, c1=-842.167, c2=-676.179 g=706.780
>871, c1=-839.988, c2=-680.924 g=701.865
>872, c1=-840.882, c2=-685.412 g=696.867
>873, c1=-841.908, c2=-692.120 g=701.219
>Saved: generated_plot_0873.png and model_0873.h5
>874, c1=-839.403, c2=-697.846 g=699.252
>875, c